In [131]:
!pip install neattext

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Import libraries.

In [132]:
import spacy
import re
import nltk
import string
import sklearn
import neattext as nt
import neattext.functions as nfx
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import AutoTokenizer, TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Read data with pandas

In [133]:
data = pd.read_csv('/kaggle/input/augmented-dataset/augmented_dataset.csv')

### Inspecting the data.

In [134]:
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [135]:
data.groupby('cyberbullying_type').describe()

tweet_text         \
                         count unique   
cyberbullying_type                      
age                      15984  15975   
ethnicity                15922  15907   
gender                   15946  15880   
not_cyberbullying        15890  15846   
other_cyberbullying      15646  15613   
religion                 15996  15989   

                                                                             
                                                                   top freq  
cyberbullying_type                                                           
age                                             high school fuck bully    4  
ethnicity                                                      #ERROR!    6  
gender               after reading all the rape and homosexual joke...    3  
not_cyberbullying                                                 ROFL    2  
other_cyberbullying                                            5/14/15    2  
religion                       not all Muslim are from Islam you idiot    4

In [136]:
data['cyberbullying_type'].value_counts()

religion               15996
age                    15984
gender                 15946
ethnicity              15922
not_cyberbullying      15890
other_cyberbullying    15646
Name: cyberbullying_type, dtype: int64

In [137]:
data['cyberbullying_type'].nunique()

6

In [138]:
data['cyberbullying_type'].count()

95384

In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95384 entries, 0 to 95383
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          95384 non-null  object
 1   cyberbullying_type  95384 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [140]:
data.shape

(95384, 2)

In [141]:
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [142]:
data.tail()

,tweet_text,cyberbullying_type
95379,"black ppl are n't expect to bash anything , de...",ethnicity
95380,Turner make not withhold his disappointment . ...,ethnicity
95381,iodine curse to God . This dense nigger bitch ...,ethnicity
95382,yea fuck you RT @ therealexel : IF YOURE angst...,ethnicity
95383,Bro . uracil get tantalum chill RT @ CHILLShra...,ethnicity


In [143]:
data.isnull().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [144]:
data.drop_duplicates(inplace=True)

In [145]:
data.shape

(95210, 2)

### Shuffle data and split it into training and test sets.

In [146]:
data_train, data_test = train_test_split(data, test_size = 0.3, random_state = 42, shuffle = True, stratify = data.cyberbullying_type)

## Clean data using neattext library.

### Remove hashtags, multiple spaces and user-handles.

In [147]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_hashtags)
data_train.head()

,tweet_text,cyberbullying_type
85583,These celebrity are like those bully inch scho...,age
82050,My ma would get vaulting_horse astatine the id...,age
64289,anyhow you agree that information_technology W...,religion
11799,RT @damong__ I'm not sexist. But fuck marvel f...,gender
28029,RT @lexi4prez: this is why Pepsi is TRASH. how...,other_cyberbullying


In [148]:
# data_train['tweet_text'] = data_train['tweet_text'].apply(lambda x: nfx.remove_custom_pattern(x, term_pattern=r'&#\$ '))

data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_userhandles)

In [149]:
data_train.tail()

,tweet_text,cyberbullying_type
72826,@ FrankAlvarez see this thread . : phosphorus ...,other_cyberbullying
65571,Hahaha rich_person you get involve inch some t...,religion
1858,Fuck you if you hit nerds :/ bully Beatdown,not_cyberbullying
93959,get on her angstrom * * -- -- > RT “ @ tayyoun...,ethnicity
62505,make @ jansportnyc just brand angstrom “ homos...,gender


In [150]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_multiple_spaces)
data_train.head()

,tweet_text,cyberbullying_type
85583,These celebrity are like those bully inch scho...,age
82050,My ma would get vaulting_horse astatine the id...,age
64289,anyhow you agree that information_technology W...,religion
11799,RT I'm not sexist. But fuck marvel for disresp...,gender
28029,RT this is why Pepsi is TRASH. how are you gon...,other_cyberbullying


In [151]:
data_train.tail()

,tweet_text,cyberbullying_type
72826,@ FrankAlvarez see this thread . : phosphorus ...,other_cyberbullying
65571,Hahaha rich_person you get involve inch some t...,religion
1858,Fuck you if you hit nerds :/ bully Beatdown,not_cyberbullying
93959,get on her angstrom * * -- -- > RT “ @ tayyoun...,ethnicity
62505,make @ jansportnyc just brand angstrom “ homos...,gender


In [152]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_stopwords)
data_train.head()

,tweet_text,cyberbullying_type
85583,celebrity like bully inch school twit child ba...,age
82050,vaulting_horse astatine idea child Washington ...,age
64289,agree information_technology Washington angstr...,religion
11799,RT I'm sexist. fuck marvel disrespecting Thor ...,gender
28029,RT Pepsi TRASH. gonna nasty product shitty ads...,other_cyberbullying


In [153]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_urls)
data_train.head()

,tweet_text,cyberbullying_type
85583,celebrity like bully inch school twit child ba...,age
82050,vaulting_horse astatine idea child Washington ...,age
64289,agree information_technology Washington angstr...,religion
11799,RT I'm sexist. fuck marvel disrespecting Thor ...,gender
28029,RT Pepsi TRASH. gonna nasty product shitty ads,other_cyberbullying


In [154]:
data_train['cyberbullying_type'].unique()

array(['age', 'religion', 'gender', 'other_cyberbullying',
       'not_cyberbullying', 'ethnicity'], dtype=object)

In [155]:
data_test['cyberbullying_type'].unique()

array(['gender', 'religion', 'age', 'other_cyberbullying',
       'not_cyberbullying', 'ethnicity'], dtype=object)

In [156]:
label_enc = LabelEncoder() 


In [157]:
data_train['cyberbullying_type'] = label_enc.fit_transform(data_train['cyberbullying_type'])
data_train.head()

,tweet_text,cyberbullying_type
85583,celebrity like bully inch school twit child ba...,0
82050,vaulting_horse astatine idea child Washington ...,0
64289,agree information_technology Washington angstr...,5
11799,RT I'm sexist. fuck marvel disrespecting Thor ...,2
28029,RT Pepsi TRASH. gonna nasty product shitty ads,4


In [158]:
data_test['cyberbullying_type'] = label_enc.transform(data_test['cyberbullying_type'])


In [159]:
data_test['cyberbullying_type'].unique()

array([2, 5, 0, 4, 3, 1])

In [160]:
data_train.dtypes

tweet_text            object
cyberbullying_type     int64
dtype: object

In [161]:
data_train['cyberbullying_type'].unique()

array([0, 5, 2, 4, 3, 1])

In [162]:
data_train['cyberbullying_type'].value_counts()

5    11192
0    11183
1    11135
2    11116
3    11092
4    10929
Name: cyberbullying_type, dtype: int64

#### Label encoder encodes data by alphabetical order. 

In [163]:
# data_train['cyberbullying_type'].value_counts()


# '''
# Cyberbullying labels are

# 0 - age
# 1 - ethnicity
# 2 - gender
# 3 - not_cyberbullying
# 4 - other_cyberbullying
# 5 - religion
# '''

### Tokenize train input text with bert's Autotokenizer that we imported earlier.

In [164]:
x_train = tokenizer(
#     text = x_train.tolist(),
    text = data_train['tweet_text'].tolist(),
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [165]:
x_train['input_ids']

<tf.Tensor: shape=(66647, 100), dtype=int32, numpy=
array([[  101, 10948,  1176, ...,     0,     0,     0],
       [  101, 13454,  1158, ...,     0,     0,     0],
       [  101,  5340,  1869, ...,     0,     0,     0],
       ...,
       [  101, 10259,  1855, ...,     0,     0,     0],
       [  101,  1126,  5700, ...,     0,     0,     0],
       [  101,   137,   179, ...,     0,     0,     0]], dtype=int32)>

### Building the model and fine-tuning the model.

The first value returned by BERT model at index 0 is the last hidden state, 1 means pooler_output
We need only the hidden state, so that we can add more layers and fine-tune the model.
We'll use functional API

In [166]:
max_len = 100


input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
input_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

# 0 is the last hidden state, 1 means pooler_output
# We need only the hidden state, so that we can add more layers and fine-tune the model.
# We'll use functional API
embeddings = bert(input_ids, attention_mask=input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(6, activation='sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

### Compile the model.

In [167]:
# Bert model requires a specific learning rate as stated in the huggingface website

optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
)

loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
global_max_pooling1d_3 (GlobalM (None, 768)          0           tf_bert_model_3[0][0]      

**Tokenize test data.**

In [168]:
x_test = tokenizer(
    text = data_test['tweet_text'].tolist(), 
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)


In [ ]:
bert_train = model.fit(
    x={'input_ids':x_train['input_ids'], 'attention_mask':x_train['attention_mask']},
    y=to_categorical(data_train.cyberbullying_type),
    validation_data=(
        {'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']},to_categorical(data_test.cyberbullying_type)
    ),
    epochs=30,
    batch_size=64
)

Epoch 1/30


/opt/conda/lib/python3.7/site-packages/keras/backend.py:4847: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


1444/2083 [===================>..........] - ETA: 6:33 - loss: 0.4981 - balanced_accuracy: 0.8011

#### Tokenize test data.

In [ ]:
# To use bert model later, we need to save both the weights and the model architecture and load them both. 
# We already saved the model architechture above when we downloaded the model.


model.save_weights('model_cyber.h5')

# To use the mode again, load it into memory
# model.load_weights('model_cyber.h5')

In [ ]:
model.load_weights('model_cyber.h5')

In [ ]:
pred_raw = model.predict({'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']})

We want to check the prediction of the first input in the test set. Bert gives the probability of each class. We'll use np.argmax to get the index oc the highest probability.

In [ ]:
pred_raw[0]

In [ ]:
y_pred = np.argmax(pred_raw, axis=1)

In [ ]:
data_test.cyberbullying_type 

#### Checking the classification report.

These values in the classification report look good.

In [ ]:
print(classification_report(data_test.cyberbullying_type, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(data_test.cyberbullying_type,y_pred))